<a href="https://colab.research.google.com/github/rohts-patil/VQA-Med-2019/blob/master/notebookz/VQA_Sentence_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 573kB 2.6MB/s 
     |████████████████████████████████| 3.7MB 29kB/s 
     |████████████████████████████████| 890kB 53.3MB/s 
     |████████████████████████████████| 1.0MB 55.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=709458ffae07de903c1cc12de92e0f51eca08929b779bf1d34af73dcf64f234a
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/VQA

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/VQA


In [0]:
import glob
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from transformers import BertTokenizer
from transformers import TFBertModel

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten

In [0]:
label_dict = {'C4_Abnormality':3,
             'C3_Organ':2,
             'C2_Plane':1,
             'C1_Modality':0}

In [0]:
path =r'train/QAPairsByCategory'

filenames = glob.glob(path + "/*.txt")

dfs = []

for filename in filenames:
    df_temp = pd.read_csv(filename,delimiter='|',names=['id','question','answer'])
    df_temp['label'] = df_temp.apply(lambda x: (filename.split('/')[-1].replace('_train.txt','')), axis=1)
    dfs.append(df_temp)

df_train = pd.concat(dfs, ignore_index=True)
df_train.drop(['id','answer'], axis = 1, inplace = True) 

In [0]:
path =r'val/QAPairsByCategory'

filenames = glob.glob(path + "/*.txt")

dfs = []

for filename in filenames:
    df_temp = pd.read_csv(filename,delimiter='|',names=['id','question','answer'])
    df_temp['label'] = df_temp.apply(lambda x: (filename.split('/')[-1].replace('_val.txt','')), axis=1)
    dfs.append(df_temp)

df_valid = pd.concat(dfs, ignore_index=True)
df_valid.drop(['id','answer'], axis = 1, inplace = True) 

In [8]:
df_train.head()

,question,label
0,what kind of image is this?,C1_Modality
1,is this a t1 weighted image?,C1_Modality
2,what type of imaging modality is used to acqui...,C1_Modality
3,is this a noncontrast mri?,C1_Modality
4,what type of image modality is this?,C1_Modality


In [9]:
df_valid.head()

,question,label
0,what imaging modality was used to take this im...,C1_Modality
1,what kind of scan is this?,C1_Modality
2,was the mri taken with contrast?,C1_Modality
3,"is this a t1 weighted, t2 weighted, or flair i...",C1_Modality
4,is this a noncontrast mri?,C1_Modality


In [0]:
#train_target = df_train.pop('label')
#dataset = tf.data.Dataset.from_tensor_slices((df_train.values, train_target.values))
#for feat, targ in dataset.take(5):
#  print ('Features: {}, Target: {}'.format(feat, targ))
#bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
#bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#example = list(dataset.__iter__())[0]
#print(example[0])
#bert_train_dataset = glue_convert_examples_to_features(dataset, bert_tokenizer, 128, 'mrpc')
#bert_train_dataset = bert_train_dataset.shuffle(100).batch(32).repeat(2)

In [0]:
bert_model_name = 'bert-base-uncased'

In [11]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
MAX_LEN = 128

In [0]:
def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                    )
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

In [13]:
input_ids_train = tokenize_sentences(df_train['question'], tokenizer, MAX_LEN)
input_ids_valid = tokenize_sentences(df_valid['question'], tokenizer, MAX_LEN)

In [0]:
input_ids_train = tf.keras.preprocessing.sequence.pad_sequences(input_ids_train, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
attention_masks_train = create_attention_masks(input_ids_train)

In [0]:
input_ids_valid = tf.keras.preprocessing.sequence.pad_sequences(input_ids_valid, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
attention_masks_valid = create_attention_masks(input_ids_valid)

In [0]:
train_size = len(df_train)
validation_size = len(df_valid)

In [0]:
BATCH_SIZE = 32
NR_EPOCHS = 1

def create_dataset(data_tuple, epochs=1, batch_size=32, buffer_size=10000, train=True):
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size)
    if train:
        dataset = dataset.prefetch(1)
    
    return dataset

train_dataset = create_dataset((input_ids_train, attention_masks_train,pd.get_dummies(df_train['label']).values), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)
validation_dataset = create_dataset((input_ids_valid, attention_masks_valid, pd.get_dummies(df_valid['label']).values), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)

In [18]:
class BertClassifier(tf.keras.Model):    
    def __init__(self, bert: TFBertModel, num_classes: int):
        super().__init__()
        self.bert = bert
        self.classifier = Dense(num_classes, activation='sigmoid')
        
    @tf.function
    def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        outputs = self.bert(input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask)
        cls_output = outputs[1]
        cls_output = self.classifier(cls_output)
                
        return cls_output

model = BertClassifier(TFBertModel.from_pretrained(bert_model_name), 4)

In [20]:
import time
from transformers import create_optimizer

label_cols = ['C1_Modality', 'C2_Plane', 'C3_Organ', 'C4_Abnormality']

steps_per_epoch = train_size // BATCH_SIZE
validation_steps = validation_size // BATCH_SIZE

# | Loss Function
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
train_loss = tf.keras.metrics.Mean(name='train_loss')
validation_loss = tf.keras.metrics.Mean(name='test_loss')

# | Optimizer (with 1-cycle-policy)
warmup_steps = steps_per_epoch // 3
total_steps = steps_per_epoch * NR_EPOCHS - warmup_steps
optimizer = create_optimizer(init_lr=2e-5, num_train_steps=total_steps, num_warmup_steps=warmup_steps)

# | Metrics
train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]

@tf.function
def train_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    with tf.GradientTape() as tape:
        predictions = model(token_ids, attention_mask=masks)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables), 1.0)

    train_loss(loss)

    for i, auc in enumerate(train_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])
        
@tf.function
def validation_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    predictions = model(token_ids, attention_mask=masks, training=False)
    v_loss = loss_object(labels, predictions)

    validation_loss(v_loss)
    for i, auc in enumerate(validation_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])
                                              
def train(model, train_dataset, val_dataset, train_steps_per_epoch, val_steps_per_epoch, epochs):
    for epoch in range(epochs):
        print('=' * 50, f"EPOCH {epoch}", '=' * 50)

        start = time.time()

        for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset, total=train_steps_per_epoch)):
            train_step(model, token_ids, masks, labels)
            if i % 1000 == 0:
                print(f'\nTrain Step: {i}, Loss: {train_loss.result()}')
                for i, label_name in enumerate(label_cols):
                    print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")
                    train_auc_metrics[i].reset_states()
        
        for i, (token_ids, masks, labels) in enumerate(tqdm(val_dataset, total=val_steps_per_epoch)):
            validation_step(model, token_ids, masks, labels)

        print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start}\n')

        for i, label_name in enumerate(label_cols):
            print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")
            validation_auc_metrics[i].reset_states()

        print('\n')

        
train(model, train_dataset, validation_dataset, train_steps_per_epoch=steps_per_epoch, val_steps_per_epoch=validation_steps, epochs=NR_EPOCHS)

================================================== EPOCH 0 ==================================================



Train Step: 0, Loss: 0.8033443689346313
C1_Modality roc_auc 0.6980676651000977
C2_Plane roc_auc 0.575757622718811
C3_Organ roc_auc 0.8140096664428711
C4_Abnormality roc_auc 0.7988505363464355





Epoch 1, Validation Loss: 0.0062827011570334435, Time: 208.35309171676636

C1_Modality roc_auc 1.0
C2_Plane roc_auc 1.0
C3_Organ roc_auc 0.9999999403953552
C4_Abnormality roc_auc 1.0


